### Importing necessary libraries

In [ ]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup

### Setting start and end date for the booking

In [ ]:
start_date = "2022-12-23"
end_date = "2022-12-24"

### Creating the URL for the booking search

In [ ]:
url = f"https://www.booking.com/searchresults.fr.html?ss=Maroc&ssne=Maroc&ssne_untouched=Maroc&efdco=1&label=gen000nr-10CAIojAFCAm1hSA1YBGhNiAEBmAEzuAEFyAEe2AED6AEB-AEBiAIBqAIBuAK_lM6cBsACAdICJDdjNzllYWVmLWExZWMtNDM3Yy1iMDYyLTY4Y2EwMTVkNmMyNNgCAeACAQ&sid=6b1171757eb9061e068c6ef46fd0af2f&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=143&dest_type=country&checkin={start_date}&checkout={end_date}&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure"

### Setting headers for the requests

In [ ]:
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
}

### Offset values for the booking search results (Number of hotels/Pages)

In [ ]:
offsets=[0,25,50,75,100,125,150,175,200,225,250,275,300,325,350,375,400,425,450,475,500]

### Extract the links of the hotel booking pages

In [ ]:
links=[]
for offset in offsets:
    #send http req (GET)
    response = requests.get(url+"&offset="+str(offset), headers=headers)
    #parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    #extract links (a_href)
    for link in soup.find_all("a", class_="e13098a59f"):
        links.append(link['href'])
        
#removing duplicated links
links = list(set(links))


### Extract the details of the hotels

In [ ]:
errors=0
hotels=[]
hotels_names=[]
for link in links:
    try:
        link_resp = requests.get(link, headers=headers)
        link_soup = BeautifulSoup(link_resp.text, 'html.parser')
        hotel={}
        hotel_name=link_soup.find("h2", class_="pp-header__title").text
    
        # Checking if the hotel has already been processed
        if hotel_name in hotels_names:
            continue
        hotels_names.append(hotel_name)
        hotel['name'] = hotel_name
        
        #geoLocation
        hotel_location = link_soup.find("a", id="hotel_sidebar_static_map")
        hotel_location = [float(s) for s in hotel_location['data-atlas-latlng'].split(',')]
        hotel["location_x"]= hotel_location[0]
        hotel["location_y"] = hotel_location[1]

        #number_of_reviews
        hotel_reviews = link_soup.find("div", class_="d8eab2cf7f c90c0a70d3 db63693c62")
        hotel["reviews"] = [int(s) for s in hotel_reviews.text.split() if s.isdigit()][0]
        
        #rating_average
        hotel["rating"] = float(link_soup.find("div", class_="b5cd09854e d10a6220b4").text.replace(",","."))
        
        #rating_by_type_of_service
        hotel_cats_rating = link_soup.find_all("div", class_="ee746850b6 b8eef6afe1")
        hotel["staff"] = float(hotel_cats_rating[0].text.replace(",","."))
        hotel["facilities"] = float(hotel_cats_rating[1].text.replace(",","."))
        hotel["cleanliness"] = float(hotel_cats_rating[2].text.replace(",","."))
        hotel["comfort"] = float(hotel_cats_rating[3].text.replace(",","."))
        hotel["money_value"] = float(hotel_cats_rating[4].text.replace(",","."))
        hotel["location"] = float(hotel_cats_rating[5].text.replace(",","."))
        hotel["wifi"] = float(hotel_cats_rating[6].text.replace(",","."))
        hotel["night_price"] = float(link_soup.find("span", class_="prco-valign-middle-helper").text[4:].replace(" ",""))

        hotels.append(hotel)
    except:
        errors+=1
        continue

### Convert the hotels list into a pandas dataframe and save it as a csv file

In [ ]:
df = pd.DataFrame(hotels)
df.to_csv('hotels.csv', index=False, header=True)